Kod referansı: https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score,cohen_kappa_score

In [2]:
#Bu, komut dosyası tutarlı tutulursa her seferinde aynı sonucu yeniden üretmek için kullanılır, 
#aksi takdirde her çalıştırma farklı sonuçlar üretir. Seed herhangi bir sayıya ayarlanabilir. 
np.random.seed(500) 

In [3]:
#veri kumesini oku
Corpus = pd.read_csv('/content/drive/MyDrive/veri/dataset.csv',encoding='utf-8-sig')

In [4]:
# Modeli Eğitim ve Test Veri kümesine ayır
#Modele uyması için eğitim veri seti kullanılacak ve tahminler test veri seti üzerinde gerçekleştirilecektir.
#Bu, sklearn kitaplığındaki train_test_split aracılığıyla yapılabilir. 
#Test_size = 0.3 parametresini ayarladığımız için Eğitim Verilerin % 70'ine sahip olacak ve Test verileri kalan %30'a sahip olacaktır.
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['comments'],Corpus['sentiments'],test_size=0.3)

In [5]:
# Etiket, hedef değişkeni kodlar
#Bu, veri kümesindeki dizi türünün Kategorik verilerini modelin anlayabileceği sayısal değerlere dönüştürmek için yapılır.
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

Metin belgelerinin bir koleksiyonunu sayısal özellik vektörlerine dönüştürmenin genel bir işlemidir. Metin verilerini modelin anlayabileceği vektörlere dönüştürmek için birçok yöntem vardır, ancak en popüler yönteme TF-IDF denir. Bu, her kelimeye atanan sonuç puanlarının bileşenleri olan "Terim Frekansı - Ters Belge" Sıklığı anlamına gelen bir kısaltmadır.
1. Terim Sıklığı: Bu, belirli bir kelimenin bir belgede ne sıklıkla göründüğünü özetler.
2. Ters Belge Frekansı: Bu, belgelerde çok görünen sözcüklerin ölçeğini küçültür.





In [6]:
#Max_features = 5000 parametresini ayarladığımız için maksimum 5000 benzersiz kelime / özellik olacaktır.
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['comments'])
#Son olarak, Train_X ve Test_X'i vektörleştirilmiş Train_X_Tfidf ve Test_X_Tfidf'e dönüştüreceğiz. 
#Bunlar artık her satır için benzersiz bir tamsayı listesi ve bunların TF-IDF tarafından hesaplanan ilişkili önemlerini içerecektir.
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

**Naive** **Bayes** **Multinomial**

In [7]:
# Eğitim veri kümesini NB sınıflandırıcısına sığdır
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# doğrulama veri kümesindeki etiketleri tahmin et
predictions_NB = Naive.predict(Test_X_Tfidf)
# Doğruluğu elde etmek için accuracy,recall,precision ve f1 score hesapla
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y))
print("Naive Bayes Precision Score -> ",precision_score(predictions_NB, Test_Y))
print("Naive Bayes Recall Score -> ",recall_score(predictions_NB, Test_Y))
print("Naive Bayes F1 Score -> ",f1_score(predictions_NB, Test_Y))
print("Naive Bayes Cohen's Kappa -> ",cohen_kappa_score(predictions_NB, Test_Y))

Naive Bayes Accuracy Score ->  0.8667562122229684
Naive Bayes Precision Score ->  0.9581206567861755
Naive Bayes Recall Score ->  0.8716083916083917
Naive Bayes F1 Score ->  0.9128193109913288
Naive Bayes Cohen's Kappa ->  0.6329644900978654


**Naive** **Bayes** **Bernoulli**


In [9]:
# Eğitim veri kümesini NB sınıflandırıcısına sığdır
Naive = naive_bayes.BernoulliNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# doğrulama veri kümesindeki etiketleri tahmin et
predictions_NB = Naive.predict(Test_X_Tfidf)
# Doğruluğu elde etmek için accuracy,recall,precision ve f1 score hesapla
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y))
print("Naive Bayes Precision Score -> ",precision_score(predictions_NB, Test_Y))
print("Naive Bayes Recall Score -> ",recall_score(predictions_NB, Test_Y))
print("Naive Bayes F1 Score -> ",f1_score(predictions_NB, Test_Y))
print("Naive Bayes Cohen's Kappa -> ",cohen_kappa_score(predictions_NB, Test_Y))

Naive Bayes Accuracy Score ->  0.8368927691963286
Naive Bayes Precision Score ->  0.9391181354160261
Naive Bayes Recall Score ->  0.8519861638027226
Naive Bayes F1 Score ->  0.8934327921603042
Naive Bayes Cohen's Kappa ->  0.549460326025178


In [15]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred=predictions_NB,y_true=Test_Y)
print('True Positives: ',cm[1,1])
print('False Positives: ',cm[0,1])
print('True Negatives: ',cm[0,0])
print('False Negatives: ',cm[1,0])

True Positives:  15271
False Positives:  2653
True Negatives:  3421
False Negatives:  990


**Naive Bayes GaussianNB**

In [16]:
# Eğitim veri kümesini NB sınıflandırıcısına sığdır
Naive = naive_bayes.GaussianNB()
Naive.fit(Train_X_Tfidf.toarray(),Train_Y)
# doğrulama veri kümesindeki etiketleri tahmin et
predictions_NB = Naive.predict(Test_X_Tfidf.toarray())
# Doğruluğu elde etmek için accuracy,recall,precision ve f1 score hesapla
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y))
print("Naive Bayes Precision Score -> ",precision_score(predictions_NB, Test_Y))
print("Naive Bayes Recall Score -> ",recall_score(predictions_NB, Test_Y))
print("Naive Bayes F1 Score -> ",f1_score(predictions_NB, Test_Y))
print("Naive Bayes Cohen's Kappa -> ",cohen_kappa_score(predictions_NB, Test_Y))

Naive Bayes Accuracy Score ->  0.8342959480635773
Naive Bayes Precision Score ->  0.9289711579853638
Naive Bayes Recall Score ->  0.8557670518921369
Naive Bayes F1 Score ->  0.890867808804883
Naive Bayes Cohen's Kappa ->  0.5492037615989585


In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred=predictions_NB,y_true=Test_Y)
print('True Positives: ',cm[1,1])
print('False Positives: ',cm[0,1])
print('True Negatives: ',cm[0,0])
print('False Negatives: ',cm[1,0])

True Positives:  15106
False Positives:  2546
True Negatives:  3528
False Negatives:  1155


**SVM Polynomial**

In [18]:
# Classifier - Algorithm - SVM
# Eğitim veri kümesini  sınıflandırıcıya sığdır
SVM = svm.SVC(C=1.0, kernel='poly', degree=3, gamma='scale')
SVM.fit(Train_X_Tfidf,Train_Y)
# doğrulama veri kümesindeki etiketleri tahmin et
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Doğruluğu elde etmek için accuracy,recall,precision ve f1 score hesapla
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y))
print("SVM  Precision Score -> ",precision_score(predictions_SVM, Test_Y))
print("SVM  Recall Score -> ",recall_score(predictions_SVM, Test_Y))
print("SVM  F1 Score -> ",f1_score(predictions_SVM, Test_Y))
print("SVM Cohen's Kappa -> ",cohen_kappa_score(predictions_SVM, Test_Y))

SVM Accuracy Score ->  0.8813521379001567
SVM  Precision Score ->  0.9280487054916672
SVM  Recall Score ->  0.91068734536238
SVM  F1 Score ->  0.9192860623781677
SVM Cohen's Kappa ->  0.6955059695291288


In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred=predictions_SVM,y_true=Test_Y)
print('True Positives: ',cm[1,1])
print('False Positives: ',cm[0,1])
print('True Negatives: ',cm[0,0])
print('False Negatives: ',cm[1,0])

True Positives:  15091
False Positives:  1480
True Negatives:  4594
False Negatives:  1170


**SVM RBF**

In [20]:
# Classifier - Algorithm - SVM
# Eğitim veri kümesini  sınıflandırıcıya sığdır
SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='scale')
SVM.fit(Train_X_Tfidf,Train_Y)
# doğrulama veri kümesindeki etiketleri tahmin et
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Doğruluğu elde etmek için accuracy,recall,precision ve f1 score hesapla
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y))
print("SVM  Precision Score -> ",precision_score(predictions_SVM, Test_Y))
print("SVM  Recall Score -> ",recall_score(predictions_SVM, Test_Y))
print("SVM  F1 Score -> ",f1_score(predictions_SVM, Test_Y))
print("SVM Cohen's Kappa -> ",cohen_kappa_score(predictions_SVM, Test_Y))

SVM Accuracy Score ->  0.9022162525184688
SVM  Precision Score ->  0.9513560051657339
SVM  Recall Score ->  0.9173931091739311
SVM  F1 Score ->  0.9340659340659341
SVM Cohen's Kappa ->  0.7451510076060603


In [21]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred=predictions_SVM,y_true=Test_Y)
print('True Positives: ',cm[1,1])
print('False Positives: ',cm[0,1])
print('True Negatives: ',cm[0,0])
print('False Negatives: ',cm[1,0])

True Positives:  15470
False Positives:  1393
True Negatives:  4681
False Negatives:  791


**SVM Sigmoid**

In [22]:
# Classifier - Algorithm - SVM
# Eğitim veri kümesini  sınıflandırıcıya sığdır
SVM = svm.SVC(C=1.0, kernel='sigmoid', degree=3, gamma='scale')
SVM.fit(Train_X_Tfidf,Train_Y)
# doğrulama veri kümesindeki etiketleri tahmin et
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Doğruluğu elde etmek için accuracy,recall,precision ve f1 score hesapla
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y))
print("SVM  Precision Score -> ",precision_score(predictions_SVM, Test_Y))
print("SVM  Recall Score -> ",recall_score(predictions_SVM, Test_Y))
print("SVM  F1 Score -> ",f1_score(predictions_SVM, Test_Y))
print("SVM Cohen's Kappa -> ",cohen_kappa_score(predictions_SVM, Test_Y))

SVM Accuracy Score ->  0.8154466084620551
SVM  Precision Score ->  0.8858003812803641
SVM  Recall Score ->  0.8641190233367328
SVM  F1 Score ->  0.8748253871849377
SVM Cohen's Kappa ->  0.5239230541495383


In [23]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred=predictions_SVM,y_true=Test_Y)
print('True Positives: ',cm[1,1])
print('False Positives: ',cm[0,1])
print('True Negatives: ',cm[0,0])
print('False Negatives: ',cm[1,0])

True Positives:  14404
False Positives:  2265
True Negatives:  3809
False Negatives:  1857


**SVM Linear**

In [24]:
# Classifier - Algorithm - SVM
# Eğitim veri kümesini  sınıflandırıcıya sığdır
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# doğrulama veri kümesindeki etiketleri tahmin et
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Doğruluğu elde etmek için accuracy,recall,precision ve f1 score hesapla
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y))
print("SVM  Precision Score -> ",precision_score(predictions_SVM, Test_Y))
print("SVM  Recall Score -> ",recall_score(predictions_SVM, Test_Y))
print("SVM  F1 Score -> ",f1_score(predictions_SVM, Test_Y))
print("SVM Cohen's Kappa -> ",cohen_kappa_score(predictions_SVM, Test_Y))

SVM Accuracy Score ->  0.8916498768748601
SVM  Precision Score ->  0.9453908123731628
SVM  Recall Score ->  0.9093759242827566
SVM  F1 Score ->  0.9270337092202858
SVM Cohen's Kappa ->  0.7169797390487501


In [25]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred=predictions_SVM,y_true=Test_Y)
print('True Positives: ',cm[1,1])
print('False Positives: ',cm[0,1])
print('True Negatives: ',cm[0,0])
print('False Negatives: ',cm[1,0])

True Positives:  15373
False Positives:  1532
True Negatives:  4542
False Negatives:  888
